In [ ]:
##################################################################################################
##################################################################################################


### This file and the source code provided can be used only for the projects and assignments  
### of this course

### Last Edit by Atef Bader: 03/22/2018


##################################################################################################

In [ ]:
import json
import requests
import datetime
import time
import os
import psycopg2
import pandas as pd
import urllib.request
import json
import csv

from pprint import pprint
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT 


In [ ]:
from elasticsearch import Elasticsearch, helpers 

es = Elasticsearch()

In [ ]:
import time

while True:
    # Get divvy statations status
    # Status is returned as a json reply
    response = urllib.request.urlopen('https://feeds.divvybikes.com/stations/stations.json')
    list__divvy_station_documents = []
    # Extract the body of the reply
    response_body = response.read()

    # Decode the format in json format
    stations_json = json.loads(response_body.decode("utf-8"))
    
    # load data file into a dataframe
    df__stations = pd.DataFrame(stations_json['stationBeanList'])
    
    # Clean the BLANKs from the data
    df__stations['altitude'] = df__stations['altitude'].apply(lambda x: 0 if x == '' else x)
    df__stations['postalCode'] = df__stations['postalCode'].apply(lambda x: 0 if x == '' else x)
    
    #Remove the whitespace from the end of hte city name
    df__stations['city'] = df__stations['city'].apply(lambda x: 'Chicago' if x == 'Chicago ' else x)

    # store the data in a file in case needed it for realtime debugging
    df__stations.to_csv('divvy_stations_logs.csv',sep=',', encoding='utf-8', index=False)
    
    for review in stations_json['stationBeanList']:
        index_review = {  
            "_index": "divvy_station_logs_es",
            "_type": "logs",
            "_id": review['id'],
            "_source": review
        }
        list__divvy_station_documents.append(index_review) 
    
    helpers.bulk(es, list__divvy_station_documents)  # To save data in elastic search
    
    output_file = open('chicago-diivy-station-for-es.json', 'w', encoding='utf-8')

    for dic in df__stations:
        json.dump(dic, output_file) 
        output_file.write("\n")
    
    # Sleep for 3 minutes; divvy updates its stations status every 2 minutes
    print('Sent Heartbeat to Divvy Servers and Going to sleep for 2 minutes now ...')
    time.sleep(120)    
    continue